In [1]:
from scrapy import Selector
import requests
import json
import pandas as pd

In [ ]:
# Id :
# Adresse :
# Latitude :
# Longitude :

In [24]:

cookies = {
    '_gid': 'GA1.2.1009969029.1770488287',
    '_ga_PTH30TD9RH': 'GS2.1.s1770490921$o4$g0$t1770490921$j60$l0$h0',
    '_ga': 'GA1.2.1501809574.1770100097',
    '_ga_6J3L27WGQH': 'GS2.2.s1770490921$o4$g0$t1770490921$j60$l0$h0',
}

headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36 OPR/126.0.0.0',
    # 'Cookie': '_gid=GA1.2.1009969029.1770488287; _ga_PTH30TD9RH=GS2.1.s1770490921$o4$g0$t1770490921$j60$l0$h0; _ga=GA1.2.1501809574.1770100097; _ga_6J3L27WGQH=GS2.2.s1770490921$o4$g0$t1770490921$j60$l0$h0',
}

response = requests.get('https://www.djezzy.dz/nos-boutiques/', headers=headers)

In [25]:
response.status_code

200

In [26]:
print(response.text)


   <!DOCTYPE html>
<html lang="fr" dir="ltr" class="ltr">
<head>
    <title>
        NOS CENTRES DE VENTE ET DE SERVICES    </title>
    <meta property="og:description" content="Djezzy, opérateur téléphonique, Leader du Digital en Algérie avec plus de 17 millions d&#039;abonnés." />
<meta property="og:title" content="NOS CENTRES DE VENTE ET DE SERVICES"/>
<meta property="og:type" content="article"/>
<meta property="og:url" content="https://www.djezzy.dz/nos-boutiques/"/>
<meta property="og:site_name" content="Djezzy"/>
<meta property="og:image" content=""/>
<link rel="image_src" href="" />

<meta name="twitter:title" content="NOS CENTRES DE VENTE ET DE SERVICES" />
<meta name="twitter:description" content="Djezzy, opérateur téléphonique, Leader du Digital en Algérie avec plus de 17 millions d&#039;abonnés." />
<meta name="twitter:image" content="" />
<meta name="twitter:card" content="summary_large_image" />
<meta name="twitter:site" content="@djezzy" />
<meta name="twitter:creator" c

In [27]:
html_sel = Selector(text=response.text)
html_sel

<Selector query=None data='<html lang="fr" dir="ltr" class="ltr"...'>

## Option 1

In [31]:
data_item = html_sel.css('div.google-maps::attr(data-items)').get()
data_item

'[{"cds":"ADRAR","code":"Z56","categorie":"C","adresse":"Groupe 74, Pr\\u00e9s souk Dinar Tayeb, Adrar.","email":"service-clientele@djezzy.dz","horaires_ouverture":"08H00-18H00","vendredi":"Ferm\\u00e9","fetes":"Ouvert","code_ouverture":"","latitude":"27.87194","longitude":"-0.28569","comment":""},{"cds":"Ain Defla - Emir Abdelkader","code":"Z53","categorie":"B","adresse":"Boulevard Emir Abdelkader, Ain Defla.","email":"service-clientele@djezzy.dz","horaires_ouverture":"08H00-18H00","vendredi":"Ferm\\u00e9e","fetes":"Ouvert","code_ouverture":"","latitude":"36.26649","longitude":"1.97418","comment":""},{"cds":"Ain Defla - Khemis Meliana","code":"Z89","categorie":"C","adresse":"Rue Belsaadi Abdelkader, Khemis Meliana, A\\u00efn Defla.","email":"service-clientele@djezzy.dz","horaires_ouverture":"08H00-19H00","vendredi":"Ferm\\u00e9e","fetes":"Ouvert","code_ouverture":"A44B002","latitude":"36.26118","longitude":"2.21461","comment":""},{"cds":"AIN TIMOUCHENT","code":"Z58","categorie":"","ad

In [32]:
items = json.loads(data_item)
items

[{'cds': 'ADRAR',
  'code': 'Z56',
  'categorie': 'C',
  'adresse': 'Groupe 74, Prés souk Dinar Tayeb, Adrar.',
  'email': 'service-clientele@djezzy.dz',
  'horaires_ouverture': '08H00-18H00',
  'vendredi': 'Fermé',
  'fetes': 'Ouvert',
  'code_ouverture': '',
  'latitude': '27.87194',
  'longitude': '-0.28569',
  'comment': ''},
 {'cds': 'Ain Defla - Emir Abdelkader',
  'code': 'Z53',
  'categorie': 'B',
  'adresse': 'Boulevard Emir Abdelkader, Ain Defla.',
  'email': 'service-clientele@djezzy.dz',
  'horaires_ouverture': '08H00-18H00',
  'vendredi': 'Fermée',
  'fetes': 'Ouvert',
  'code_ouverture': '',
  'latitude': '36.26649',
  'longitude': '1.97418',
  'comment': ''},
 {'cds': 'Ain Defla - Khemis Meliana',
  'code': 'Z89',
  'categorie': 'C',
  'adresse': 'Rue Belsaadi Abdelkader, Khemis Meliana, Aïn Defla.',
  'email': 'service-clientele@djezzy.dz',
  'horaires_ouverture': '08H00-19H00',
  'vendredi': 'Fermée',
  'fetes': 'Ouvert',
  'code_ouverture': 'A44B002',
  'latitude': '3

In [ ]:
for item in items:
    print(item['cds'])
    print(item['adresse'])
    print(item['latitude'])
    print(item['longitude'])


ADRAR
Groupe 74, Prés souk Dinar Tayeb, Adrar.
27.87194
-0.28569
Ain Defla - Emir Abdelkader
Boulevard Emir Abdelkader, Ain Defla.
36.26649
1.97418
Ain Defla - Khemis Meliana
Rue Belsaadi Abdelkader, Khemis Meliana, Aïn Defla.
36.26118
2.21461
AIN TIMOUCHENT
Boulevard du 1er Novembre, N° 19, Ain Temouchent.
35.29095
-1.14046
Alger - Audin
23, Rue Didouche Mourad, Place Audin, Alger.
36.76921
3.05506
Alger - Bab El Oued
06, Boulevard Abderrahmane Mira, BEO, Alger.
36.79212
3.05714
Alger - Bab Ezzouar / Centre commercial
Centre commercial et des loisirs -Quartier des affaires- Bab Ezzouar, Alger.
36.71258
3.1966
Alger - Bir Mourad Rais
Rue Saada Larbi (Ex Gambita) Groupe 37 N° 10.
36.739521
3.04958
Alger - Birkhadem
Place Taleb Mohamed Ameziane, Bir khadem, Alger
36.71569
3.0493
Alger - Dar El Beida
Rue Mohamed KHEMISTI N°14 & N°16 Lot N°18 propriété N°13 – Commune Dar El Beida, Alger. 
36.71529
3.20854
Alger - Draria
573 Lot Guéssas, Route de Baba Hassen, Draria - Alger
36.71011
2.99773

In [37]:
pd.DataFrame(items).to_excel('locations.xlsx', index=False)

## Option 2

In [44]:
div_items2 = html_sel.css('div.js-cds-items.items a').getall()


In [46]:
item = div_items2[0]
item

'<a data-markerid="0" href="#" class="item marker-link">\n                            <h3>ADRAR</h3>\n                            <strong>Adresse</strong>\n                            <address>\n                                Groupe 74, Prés souk Dinar Tayeb, Adrar.                            </address>\n                        </a>'

In [49]:
item_sel = Selector(text=item)
item_sel

<Selector query=None data='<html><body><a data-markerid="0" href...'>

In [ ]:
item_sel = Selector(text=item)
cds = item_sel.css('h3::text').get().strip()
adresse = item_sel.css('address::text').get().strip()

print(cds)
print(adresse)

ADRAR
Groupe 74, Prés souk Dinar Tayeb, Adrar.


In [52]:
final_data = []

for item in div_items2:
    item_sel = Selector(text=item)
    cds = item_sel.css('h3::text').get().strip()
    adresse = item_sel.css('address::text').get().strip()
    final_data.append({'cds': cds, 'adresse': adresse})



In [53]:
pd.DataFrame(final_data).to_excel('locations2.xlsx', index=False)